In [3]:
import time
from selenium import webdriver
import pandas as pd
import re
import datetime
from collections import OrderedDict

In [6]:
class NaverNewsCrawler(object):
    
    '''
    입력한 키워드로 검색한 뉴스를 크롤링하는 클래스입니다.
    제목, 본문요약, 등록일, 언론사명를 기록합니다.
    '''
    
    
    def __init__(self, keyword):
        
        self.keyword = keyword
        self.now = datetime.datetime.now()

        self.item = OrderedDict(\
                       {'title' : [],
                        'text' : [],
                        'date' : [],
                        'publisher' : []
                       })
        
        
        # ?분 전, ?시간 전, ?일 전으로 표시되는 날짜를 정확히 계산하기 위한 정규표현식
        self.p_today = re.compile('.시간|.분')
        self.p_days_age = re.compile('.일 전')
        self.p_date = re.compile('[0-9]*\.[0-9]*\.[0-9]*')
        
        
        # Driver 열기
        path = "D:\chromedriver.exe"
        self.driver = webdriver.Chrome(path)
        self.driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%s"%keyword)
        time.sleep(3)
        
        
    
    
    
    
    # 크롤링한 title에서 특수문자를 제거하는 함수입니다.
    def _clean_title(self,text):
        text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '' , text)
        return text
    
    
    
    
    
    
    # 크롤링한 text에서 불필요한 부분들을 제거하는 함수입니다.
    def _clean_text(self,text):
        
        # () 와 []로 감싸진 부분들은 전부 제거
        text = re.sub('\(.+?\)', '',string = text)
        text = re.sub('\[.+?\]', '',string = text)
        text = re.sub('◀.+?▶', '',string = text)
        text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]▲', '' , text) # 특수문자 제거
        text = re.sub('... 기자', '', text)   # 기자라는 단어도 제거
        
        return text
        
        
        
        
        
    # 크롤링한 날짜를 YYYYmmdd 형식으로 반환하는 함수입니다.
    def _clean_date(self, text):
        
        # ?분 전, ?시간 전으로 표시되어 있다면 오늘 날짜로 치환
        if self.p_today.search(text):
            date = self.now.strftime("%Y%m%d")
        
        # ?일 전으로 표시되어 있다면 n일 전의 날짜로 치환
        elif self.p_days_age.search(text):
            n = self.p_days_age.search(text).group()[0] # n일 전의 n을 추출
            
            date = self.now - datetime.timedelta(days = int(n))
            date = date.strftime("%Y%m%d")
            
        else:
            date = self.p_date.search(text).group().replace('.','')
            
        return date
            
        
        
        
        
    # 크롤링한 publisher에서 불필요한 단어들을 제거하는 함수입니다.
    def _clean_publisher(self, text):
        return text.replace('언론사', '').replace('선정', '').replace('  ', '')
    
    
    
    
    
    
    
    
    
    # 타이틀을 크롤링하는 함수입니다.
    def _crawl_title(self):
        title_ls = [self._clean_title(title.text) for title in self.driver.find_elements_by_class_name(' _sp_each_title')]
        return title_ls
        
        
    # 본문 요약(text)을 크롤링하는 함수입니다.
    def _crawl_text(self):
        text_ls = [self._clean_text(text.text) for text in self.driver.find_elements_by_css_selector('dl > dd:nth-child(3)') if not text.text == '']
        return text_ls
    
    
    # 날짜를 크롤링하는 함수입니다.
    def _crawl_date(self):
        date_ls = [self._clean_date(date.text) for date in self.driver.find_elements_by_css_selector('dl > dd.txt_inline')]
        return date_ls
        
        
    # 언론사명를 크롤링하는 함수입니다.
    def _crawl_publisher(self):
        publisher_ls = [self._clean_publisher(title.text) for title in self.driver.find_elements_by_class_name(' _sp_each_source')]
        return publisher_ls
    
    
    
    def crawl_process(self, n_page):
        self.item = OrderedDict(\
                       {'title' : [],
                        'text' : [],
                        'date' : [],
                        'publisher' : []
                       })
        
        for page_cnt in range(n_page):
            
            # 크롤링을 수행합니다.
            self.item['title'] += self._crawl_title()
            self.item['text'] += self._crawl_text()
            self.item['date'] += self._crawl_date()
            self.item['publisher'] += self._crawl_publisher()
            
            
            # 다음 페이지로 넘어갑니다.
            try:
                self.driver.find_element_by_css_selector('#main_pack > div.news.mynews.section._prs_nws > div.paging > a.next').click()
                time.sleep(1)
            except:
                return self.item
        
        return self.item

In [7]:
my_crawler = NaverNewsCrawler('삼성전자')

In [8]:
temp_dict = my_crawler.crawl_process(n_page= 5)

In [9]:
for key in temp_dict.keys():
    print(key, len(temp_dict[key]))

title 50
text 50
date 50
publisher 50


In [10]:
data_df = pd.DataFrame(temp_dict)

In [11]:
data_df

,title,text,date,publisher
0,삼성 소프트웨어 인재 이어 창업 인재 키운다…500개 스타트업 육성,삼성전자가 ‘창업 인재’ 지원에 나선다. 사내 직원을 대상으로 운영하던 창업 지원 ...,20181017,중앙일보
1,3D 프린터 활용하는 삼성전자 C랩 과제원들,17일 서울대학교 내 삼성전자-서울대 공동연구소 C랩 라운지에서 C랩 과제원들이 ...,20181017,연합뉴스
2,C랩 성과 발표하는 이재일 삼성전자 상무,17일 서울대학교 내 삼성전자-서울대 공동연구소 C랩에서 삼성전자 창의개발센터 이...,20181017,연합뉴스
3,데일리안 오늘뉴스 종합평양행 초청장 전달 위해바티칸에 공들인 文대,"▲삼성전자, 차세대 네트워크 서비스 분석 솔루션 기업 지랩스 인수 삼성전자는 차세대...",20181017,데일리안
4,삼성전자 CO₂누출사고 원인 삼성하청업체 진술 엇갈린다,이재명 경기도지사가 17일 자신의 페이스북에 이날 오전 있었던 삼성전자 이산화탄소 ...,20181017,국제뉴스
5,삼성전자 차량용 반도체 브랜드 Exynos Auto · ISOCELL Auto 출시,머니투데이 더리더 박영복 삼성전자가 자동차용 프로세서 브랜드 'Exynos Auto...,20181017,더리더
6,누출 사고 삼성전자 기흥사업장 검찰 송치,삼성전자 기흥사업장 이산화탄소 누출 사고에 대한 중간 조사 결과가 발표됐습니다....,20181017,MBC
7,에이스침대 삼성전자와 콜라보 행사 하이마트쇼핑몰서 진행,"‘건강한 침실 공간’, ‘우리 아이 위한 침실’, ‘프리미엄 힐링’ 세 가지 테마로...",20181017,아시아투데이
8,경기도 이산화탄소 누출 사고 삼성전자 기흥사업장 소방시설법 위반으로 검,사진은 지난 9월4일 사고가 발생한 삼성 기흥 반도체 공장 정문으로 삼성전자 자체 ...,20181017,경인일보
9,삼성電 스페인 지랩스 인수5G 장비 강화,AI 기반 통신 품질 측정·분석 강화에 초점 삼성전자가 네트워크 서비스 분석 업체를...,20181017,ZDNet Korea
